In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import json

import librosa
import librosa.display
import IPython.display as ipd

import warnings
warnings.filterwarnings('ignore')

from pydub import AudioSegment
#import tensorflow.keras as keras

import os
import re
import csv

import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance

In [2]:
#List of genres in the dataset
print(list(os.listdir("/Users/anveshvarma/Downloads/ADM_MUSIC/genres_original")))

['pop', '.DS_Store', 'metal', 'disco', 'blues', 'reggae', 'classical', 'rock', 'hiphop', 'country', 'jazz']


In [3]:
dataset_path= "/Users/anveshvarma/Downloads/ADM_MUSIC/genres_original"


In [4]:
if dataset_path is not dataset_path:
    print(1)

In [ ]:
data_soundwaves=[]
for i, (dirpath, dirnames, filenames) in enumerate(os.walk("/Users/anveshvarma/Downloads/ADM_MUSIC/genres_original")):
    if(i!=0):
        print(dirpath)
        for f in filenames:
            print(f)
            file_path = os.path.join(dirpath, f)
            signal, sr = librosa.load(file_path)
            data_soundwaves.append([re.split(r'[_.]', f)[0],f,signal, sr])

In [10]:
df_soundwaves = pd.DataFrame(data_soundwaves, columns = ['genre_name','filename','sound_wave','sr'])
df_soundwaves

genre_name        filename  \
0          pop   pop.00027.wav   
1          pop   pop.00033.wav   
2          pop   pop.00032.wav   
3          pop   pop.00026.wav   
4          pop   pop.00030.wav   
..         ...             ...   
908       jazz  jazz.00014.wav   
909       jazz  jazz.00000.wav   
910       jazz  jazz.00038.wav   
911       jazz  jazz.00004.wav   
912       jazz  jazz.00010.wav   

                                            sound_wave     sr  
0    [0.03451538, 0.04815674, 0.06430054, 0.0982055...  22050  
1    [-0.0019836426, 0.06552124, 0.1253357, 0.11224...  22050  
2    [0.012359619, -0.0107421875, -0.03161621, -0.0...  22050  
3    [-0.004119873, -0.013946533, -0.009338379, -0....  22050  
4    [-0.11010742, -0.15396118, 0.025970459, 0.1386...  22050  
..                                                 ...    ...  
908  [-0.15530396, -0.21246338, -0.08627319, -0.020...  22050  
909  [-0.009246826, -0.011779785, -0.013702393, -0....  22050  
910  [-0.20962524, -0.3809204, -0.39624023, -0.4165...  22050  
911  [0.122039795, 0.22442627, 0.19665527, 0.171508...  22050  
912  [0.008270264, 0.010284424, 0.0060424805, 0.005...  22050  

[913 rows x 4 columns]

In [11]:
data_soundwaves_single=df_soundwaves.to_numpy()
data_soundwaves_single

array([['pop', 'pop.00027.wav',
        array([ 0.03451538,  0.04815674,  0.06430054, ..., -0.06954956,
               -0.1444397 , -0.19223022], dtype=float32)               ,
        22050],
       ['pop', 'pop.00033.wav',
        array([-0.00198364,  0.06552124,  0.1253357 , ...,  0.22796631,
                0.31658936,  0.3052063 ], dtype=float32)               ,
        22050],
       ['pop', 'pop.00032.wav',
        array([ 0.01235962, -0.01074219, -0.03161621, ..., -0.19876099,
                0.03173828,  0.05938721], dtype=float32)               ,
        22050],
       ...,
       ['jazz', 'jazz.00038.wav',
        array([-0.20962524, -0.3809204 , -0.39624023, ...,  0.03085327,
                0.02249146,  0.00622559], dtype=float32)               ,
        22050],
       ['jazz', 'jazz.00004.wav',
        array([0.12203979, 0.22442627, 0.19665527, ..., 0.0116272 , 0.03677368,
               0.05535889], dtype=float32)                                     ,
        22050],
   

In [12]:
def waveplot(sound_wave):
    plt.figure(figsize = (16, 6))
    plt.title(sound_wave[0])
    librosa.display.waveshow(y=sound_wave[2], sr=sound_wave[3], color="#2f7d92ff")
    plt.show()

In [ ]:
for sound in data_soundwaves_single:
    waveplot(sound)

In [ ]:
def hermony_plot(sound_wave):
    plt.figure(figsize = (16, 6))
    plt.title(sound_wave[0])
    y_harm, y_perc = librosa.effects.hpss(sound_wave[2])
    plt.plot(y_harm, color = '#A300F9');
    plt.plot(y_perc, color = '#FFB100');
    plt.show()

In [ ]:
for sound in data_soundwaves_single:
    hermony_plot(sound)

In [ ]:
hop_length = 5000
data=[]
dataset_path = "/Users/anveshvarma/Downloads/ADM_MUSIC/genres_original"
for i, (dirpath, dirnames, filenames) in enumerate(os.walk("/Users/anveshvarma/Downloads/ADM_MUSIC/genres_original")):
     if dirpath is not dataset_path:
            for f in filenames:
                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path)
                zero_crossings = librosa.zero_crossings(signal, pad=False)
                data.append([re.split(r'[_.]', f)[0],f,sum(zero_crossings)])

In [ ]:
data_zerocrossings=[]
for soundwave in data_soundwaves:
    zero_crossings = librosa.zero_crossings(soundwave[2], pad=False)
    data_zerocrossings.append([soundwave[0],soundwave[1],sum(zero_crossings)])

In [ ]:
df_zerocrossings = pd.DataFrame(data_zerocrossings)
df_zerocrossings

In [ ]:
df_zerocrossings = df_zerocrossings.sort_values(by=[2])
set1 = set(df_zerocrossings[0])
plt.figure(figsize=(10,8))

for count in set1:
     selected_data = df_zerocrossings.loc[df_zerocrossings[0] == count]
     plt.plot(np.linspace(0, len(selected_data), len(selected_data), endpoint=False), selected_data[2], label=count)
    
plt.legend()
plt.show()

In [ ]:
classical_song_path='/Users/anveshvarma/Downloads/ADM_MUSIC/genres_original/classical/classical.00003.wav'
classical_sound, classical_sr =  librosa.load(classical_song_path)

mfccs = librosa.feature.mfcc(classical_sound, sr=sr)
mfccs = sklearn.preprocessing.scale(mfccs, axis=1)
print('Mean:', mfccs.mean(), '\n')
print('Var:', mfccs.var())

#Displaying  the MFCCs:
plt.figure(figsize = (16, 6))
librosa.display.specshow(mfccs, sr=sr, x_axis='time', cmap = 'cividis');

In [ ]:
path =".\\three_second_songs"
try: 
    os.mkdir(path) 
except OSError as error: 
    print(error)  

In [ ]:
hop_length = 5000
dataset_path = ".\\Dataset\\Data\\genres_original"
three_second_folder = ".\\three_second_songs"
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
     if dirpath is not dataset_path:
            for f in filenames:
                file_path = os.path.join(dirpath, f)
                for i in range(0,29,3):
                    t1 = i * 1000 
                    t2 = (i+3)* 1000
                    newAudio = AudioSegment.from_wav(file_path)
                    newAudio = newAudio[t1:t2]
                    split_fn = three_second_folder +"\\" + str(i)+'_'+f 
                    newAudio.export(split_fn, format="wav")

In [ ]:
def feature_extraction(dataset_path, csv_file, hop_length):

    data=[]
    mfcc_mean=[]
    mfcc_var=[]
    hop_length=hop_length

    header = ['filename','chromagram_mean','chromagram_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','spectral_rolloff_mean','spectral_rolloff_var','zero_crossing_rate_mean','zero_crossing_rate_var','tempo_mean','y_harm_mean','y_perc_mean','y_harm_var','y_perc_var','mfccm_1','mfccm_2','mfccm_3','mfccm_4','mfccm_5','mfccm_6','mfccm_7','mfccm_8','mfccm_9','mfccm_10','mfccm_11','mfccm_12','mfccm_13','mfccm_14','mfccm_15','mfccm_16','mfccm_17','mfccm_18','mfccm_19','mfccm_20','mfccv_1','mfccv_2','mfccv_3','mfccv_4','mfccv_5','mfccv_6','mfccv_7','mfccv_8','mfccv_9','mfccv_10','mfccv_11','mfccv_12','mfccv_13','mfccv_14','mfccv_15','mfccv_16','mfccv_17','mfccv_18','mfccv_19','mfccv_20']
    file = open(csv_file, 'w', newline='')

    with file:
        writer = csv.writer(file)
        writer.writerow(header)

    dataset_path = dataset_path
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(".\\three_second_songs")):
         if dirpath is not dataset_path:
                for f in filenames:
                    file_path = os.path.join(dirpath, f)
                    signal, sr = librosa.load(file_path)
                    chromagram =librosa.feature.chroma_stft(y=signal, sr=sr, hop_length=hop_length)
                    rms =librosa.feature.rms(y=signal, hop_length=hop_length)
                    spectral_centroid =librosa.feature.spectral_centroid(y=signal, sr=sr,hop_length=hop_length)
                    spectral_bandwidth=librosa.feature.spectral_bandwidth(y=signal, sr=sr,hop_length=hop_length)
                    spectral_rolloff=librosa.feature.spectral_rolloff(y=signal, sr=sr,hop_length=hop_length)
                    zero_crossing_rate=librosa.feature.zero_crossing_rate(y=signal,hop_length=hop_length)
                    tempo = librosa.beat.tempo(y=signal)
                    y_harm, y_perc = librosa.effects.hpss(signal)
                    mfcc = librosa.feature.mfcc(y=signal,hop_length=hop_length)
                    mfccm=mfcc.mean(axis=1)
                    mfccv=mfcc.var(axis=1)

                    data=[re.split(r'[_.]', f)[1],np.mean(chromagram),np.var(chromagram),np.mean(rms),np.var(rms),np.mean(spectral_centroid),
                                 np.var(spectral_centroid),np.mean(spectral_bandwidth),np.var(spectral_bandwidth),np.mean(spectral_rolloff),
                                 np.var(spectral_rolloff),np.mean(zero_crossing_rate),np.var(zero_crossing_rate),np.mean(tempo),
                                 np.mean(y_harm),np.mean(y_perc),np.var(y_harm),np.var(y_perc)]+ mfccm.tolist()+mfccv.tolist()
                    file = open('.\\Dataset\\data_f2.csv', 'a', newline='')

                    with file:
                        writer = csv.writer(file)
                        writer.writerow(data)

In [ ]:
feature_extraction( dataset_path=".\\three_second_songs", csv_file= ".\\Dataset\\data_f2.csv", hop_length = 5000) 

In [ ]:
data = pd.read_csv('.\\Dataset\\data_f2.csv') 
data.head()

In [ ]:
y = data['filename']
X = data.loc[:, data.columns != 'filename']

In [ ]:
cols = X.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(np_scaled, columns = cols)
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
model_assessment(xgb, "XGBoost")

In [ ]:
def xgbclassifier(n_estimators,learning_rate):
    xgb = XGBClassifier(n_estimators, learning_rate)
    xgb.fit(X_train, y_train)
    preds = xgb.predict(X_test)
    print('For Learning rate ', ':',learning_rate, 'and n_estimators :',n_estimators, '\n')
    print('Accuracy', ':', round(accuracy_score(y_test, preds), 4), '\n')
    confusion_matr = confusion_matrix(y_test, preds) 
    plt.figure(figsize = (16, 9))
    sns.heatmap(confusion_matr, cmap="Blues", annot=True, 
                xticklabels = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"],
               yticklabels=["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]);
    plt.savefig("conf matrix")

In [ ]:
xgbclassifier(n_estimators=1000, learning_rate=0.005)

In [ ]:
xgbclassifier(n_estimators=1000, learning_rate=0.05)

In [ ]:
xgbclassifier(n_estimators=100, learning_rate=0.05)

In [ ]:
xgbclassifier(n_estimators=1000, learning_rate=0.01)

Implementing Cosine Similarity